In [6]:
import os
import shutil
from tqdm import tqdm

In [7]:
train_folder = "/Users/monicagullapalli/Downloads/emotion_detection/train"
validation_folder = "/Users/monicagullapalli/Downloads/emotion_detection/validation"

In [8]:
combined_folder = "//Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/combined"

In [10]:
def copy_files_with_labels(source_folder, destination_folder):
    for label in os.listdir(source_folder):
        label_folder = os.path.join(source_folder, label)
        if os.path.isdir(label_folder):
            destination_label_folder = os.path.join(destination_folder, label)
            if not os.path.exists(destination_label_folder):
                os.makedirs(destination_label_folder)
            for file in tqdm(os.listdir(label_folder), desc=f"Copying {label}"):
                shutil.copy(os.path.join(label_folder, file), destination_label_folder)


copy_files_with_labels(train_folder, combined_folder)


copy_files_with_labels(validation_folder, combined_folder)

print("Data combined successfully.")

Copying Disgust: 100%|██████████| 55/55 [00:00<00:00, 3862.35it/s]

Data combined successfully.


In [19]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax') 
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    '/Users/monicagullapalli/Downloads/emotion_detection/train',
    target_size=(64, 64), 
    batch_size=32,
    class_mode='sparse'  
)


validation_generator = datagen.flow_from_directory(
    '/Users/monicagullapalli/Downloads/emotion_detection/validation',
    target_size=(64, 64), 
    batch_size=32,
    class_mode='sparse' 
)


model.fit(train_generator, validation_data=validation_generator, epochs=10)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Epoch 1/10
898/898 [==============================] - 30s 33ms/step - loss: 1.6012 - accuracy: 0.3796 - val_loss: 1.5833 - val_accuracy: 0.3920
Epoch 2/10
898/898 [==============================] - 29s 32ms/step - loss: 1.4010 - accuracy: 0.4633 - val_loss: 1.5506 - val_accuracy: 0.4191
Epoch 3/10
898/898 [==============================] - 31s 35ms/step - loss: 1.2733 - accuracy: 0.5206 - val_loss: 1.5977 - val_accuracy: 0.4074
Epoch 4/10
898/898 [==============================] - 31s 34ms/step - loss: 1.1528 - accuracy: 0.5703 - val_loss: 1.6552 - val_accuracy: 0.4152
Epoch 5/10
898/898 [==============================] - 29s 32ms/step - loss: 1.0282 - accuracy: 0.6216 - val_loss: 1.7287 - val_accuracy: 0.4274
Epoch 6/10
898/898 [==============================] - 30s 33ms/step - loss: 0.9000 - accuracy: 0.6708 - val_loss: 1.8465 - val_accuracy: 0.4227
Epoch 7/10
898/898 [==============================]

In [20]:
model.save('/Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/final_model.h5')

/Users/monicagullapalli/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


model = load_model('/Users/monicagullapalli/Downloads/Neural-networks-assignments/Neural-Networks-project/final_model.h5')  


cap = cv2.VideoCapture(0)  

class_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprise"}


while True:
    ret, frame = cap.read()  
    processed_frame = cv2.resize(frame, (64, 64))  
    processed_frame = processed_frame / 255.0 


    input_data = np.expand_dims(processed_frame, axis=0)


    predictions = model.predict(input_data)
    predicted_class = np.argmax(predictions)


    predicted_label = class_labels[predicted_class]


    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Camera', frame)

    # Additional actions based on the predicted class label
    if predicted_label == 'Angry':
        # Perform action for Angry
        print("Perform action for Angry")

    elif predicted_label == 'Sad':
        # Perform action for Sad
        print("Perform action for Sad")

    elif predicted_label == 'Happy':
        # Perform action for Happy
        print("Perform action for Happy")

    # Wait for the 'q' key to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 13ms/step
Perform action for Happy
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 14ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 14ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry
1/1 [==============================] - 0s 13ms/step
Perform action for Angry

KeyboardInterrupt: 

: 